## Import library


In [1]:
import re
import json
from pprint import pprint
import re 
import math
import json
import spacy
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from spacy.tokens import DocBin
from spacy.util import filter_spans

## Convert data from Dataturks to SpaCy

In [4]:
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    training_data = []
    lines=[]
    with open(dataturks_JSON_FilePath, 'r') as f:
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        text = data['content'].replace("\n", " ")
        entities = []
        data_annotations = data['annotation']
        if data_annotations is not None:
            for annotation in data_annotations:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    point_start = point['start']
                    point_end = point['end']
                    point_text = point['text']

                    lstrip_diff = len(point_text) - len(point_text.lstrip())
                    rstrip_diff = len(point_text) - len(point_text.rstrip())
                    if lstrip_diff != 0:
                        point_start = point_start + lstrip_diff
                    if rstrip_diff != 0:
                        point_end = point_end - rstrip_diff
                    entities.append((point_start, point_end + 1 , label))
        training_data.append((text, {"entities" : entities}))
    return training_data

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data

## Analysis the Dataset

In [5]:
dataset_path='data/Entity Recognition in Resumes.json'
data = trim_entity_spans(convert_dataturks_to_spacy(dataset_path))
pprint(data[0])

['Abhishek Jha Application Development Associate - Accenture  Bengaluru, '
 'Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a  '
 '• To work for an organization which provides me the opportunity to improve '
 "my skills and knowledge for my individual and company's growth in best "
 'possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK '
 'EXPERIENCE  Application Development Associate  Accenture -  November 2017 to '
 'Present  Role: Currently working on Chat-bot. Developing Backend Oracle '
 'PeopleSoft Queries for the Bot which will be triggered based on given input. '
 'Also, Training the bot for different possible utterances (Both positive and '
 'negative), which will be given as input by the user.  EDUCATION  B.E in '
 'Information science and engineering  B.v.b college of engineering and '
 'technology -  Hubli, Karnataka  August 2013 to June 2017  12th in '
 'Mathematics  Woodbine modern school  April 2011 to March 2013  10th  '
 'Kendriy

In [6]:
print("Total dataset: ",len(data))

Total dataset:  220


In [24]:
print('Sample resume texts:')
pprint(data[0][0])

Sample resume texts:
('Abhishek Jha Application Development Associate - Accenture  Bengaluru, '
 'Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a  '
 '• To work for an organization which provides me the opportunity to improve '
 "my skills and knowledge for my individual and company's growth in best "
 'possible ways.  Willing to relocate to: Bangalore, Karnataka  WORK '
 'EXPERIENCE  Application Development Associate  Accenture -  November 2017 to '
 'Present  Role: Currently working on Chat-bot. Developing Backend Oracle '
 'PeopleSoft Queries for the Bot which will be triggered based on given input. '
 'Also, Training the bot for different possible utterances (Both positive and '
 'negative), which will be given as input by the user.  EDUCATION  B.E in '
 'Information science and engineering  B.v.b college of engineering and '
 'technology -  Hubli, Karnataka  August 2013 to June 2017  12th in '
 'Mathematics  Woodbine modern school  April 2011 to March 20

In [31]:
print("Entity with start and end position in the text:")
pprint(data[0][1])

Entity with start and end position in the text:
{'entities': [[1296, 1622, 'Skills'],
              [993, 1154, 'Skills'],
              [939, 957, 'College Name'],
              [883, 905, 'College Name'],
              [856, 860, 'Graduation Year'],
              [771, 814, 'College Name'],
              [727, 769, 'Designation'],
              [407, 416, 'Companies worked at'],
              [372, 405, 'Designation'],
              [95, 145, 'Email Address'],
              [60, 69, 'Location'],
              [49, 58, 'Companies worked at'],
              [13, 46, 'Designation'],
              [0, 12, 'Name']]}


In [61]:
for index,person in enumerate(data[:3]):
    print("person number: ",index+1)
    for entity in person[1]['entities']:
        print(entity[2])
    print('\n')


person number:  1
Skills
Skills
College Name
College Name
Graduation Year
College Name
Designation
Companies worked at
Designation
Email Address
Location
Companies worked at
Designation
Name


person number:  2
Email Address
Skills
Graduation Year
College Name
Degree
Graduation Year
College Name
Degree
Email Address
Location
Name


person number:  3
Skills
Skills
Skills
Skills
Skills
Skills
Skills
Skills
Skills
College Name
Degree
Location
Companies worked at
Designation
Location
Companies worked at
Designation
Email Address
Location
Name




### Train Test Split

In [7]:
import random
import math

def train_test_split(data, test_size, random_state):

    random.Random(random_state).shuffle(data)
    test_idx = len(data) - math.floor(test_size * len(data))
    train_set = data[0: test_idx]
    test_set = data[test_idx: ]

    return train_set, test_set

In [8]:
train_data, test_data = train_test_split(data, test_size = 0.1, random_state = 42)
print('train set len: ',len(train_data))
print('test set len: ',len(test_data))

train set len:  198
test set len:  22


## Dump the JSON data

In [66]:
json.dump(train_data, open('data/train_data.json', 'w'))
json.dump(test_data, open('data/test_data.json', 'w'))

In [2]:
import json
import re
import math
import spacy
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from spacy.tokens import DocBin
from spacy.util import filter_spans

def preprocess_dataturks_json(dataturks_JSON_FilePath):
    training_data = []
    lines=[]
    with open(dataturks_JSON_FilePath, 'r',encoding='utf-8') as f:
        lines = f.readlines()
    for line in lines:
        data = json.loads(line)
        text = data['content'].replace("\n", " ")
        entities = []
        data_annotations = data['annotation']
        if data_annotations is not None:
            for annotation in data_annotations:
                point = annotation['points']
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]
                for label in labels:
                    point_start = point['start']
                    point_end = point['end']
                    point_text = point['text']
                    lstrip_diff = len(point_text) - len(point_text.lstrip())
                    rstrip_diff = len(point_text) - len(point_text.rstrip())
                    if lstrip_diff != 0:
                        point_start = point_start + lstrip_diff
                    if rstrip_diff != 0:
                        point_end = point_end - rstrip_diff
                    entities.append((point_start, point_end + 1 , label))
            training_data.append((text, {"entities" : entities}))
    return training_data

def trim_entity_spans(data: list) -> list:
    invalid_span_tokens = re.compile(r'\s')
    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data

def get_spacy_doc(file, data):
    nlp = spacy.blank("en")
    db = DocBin()
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']
        ents = []
        entity_indices = []
        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity = True
                    break
            if skip_entity == True:
                continue
            entity_indices = entity_indices + list(range(start, end))
            try:
                span = doc.char_span(start, end, label = label, alignment_mode = "strict")
            except:
                continue
            if span is None:
                err_data = str([start, end])+ " " + str(text) + "\n"
                file.write(err_data)
            else:
                ents.append(span)
        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass
    return db

def train_test_split(data, test_size, random_state):
    random.Random(random_state).shuffle(data)
    test_idx = len(data) - math.floor(test_size * len(data))
    train_set = data[0: test_idx]
    test_set = data[test_idx: ]
    return train_set, test_set

def convert_dataturk_json_to_spacy_data_format(train_data, test_data,spacy_dir):
    file = open("sample.txt", "w",encoding='utf-8')
    db = get_spacy_doc(file, train_data)
    train_spacy_path=spacy_dir+'/train_data.spacy'
    db.to_disk(train_spacy_path)


    db = get_spacy_doc(file, test_data)
    test_spacy_path=spacy_dir+'/test_data.spacy'
    db.to_disk(test_spacy_path)
    file.close()

if __name__=="__main__":
    dataset_path = "data/jsons/Entity Recognition in Resumes.json"
    spacy_dir='data/spacy_format'
    dataset=trim_entity_spans(preprocess_dataturks_json(dataset_path))
    train_data, test_data = train_test_split(dataset, test_size = 0.1, random_state = 42)
    convert_dataturk_json_to_spacy_data_format(train_data, test_data,spacy_dir)

In [3]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
file = open("error.txt", "w")
db = get_spacy_doc(file, train_data)
db.to_disk("spacy_data/train_data.spacy")

db = get_spacy_doc(file, test_data)
db.to_disk("spacy_data/test_data.spacy")
file.close()

100%|██████████| 22/22 [00:00<00:00, 28.71it/s]


In [11]:
!python -m spacy train config.cfg --output ./output --paths.train spacy_data/train_data.spacy --paths.dev spacy_data/test_data.spacy 

ℹ Saving to output directory: output

In [12]:
nlp = spacy.load("output/model-best")

doc = nlp("Harini Komaravelli Test Analyst at Oracle, Hyderabad  Hyderabad, Telangana - Email me on Indeed: indeed.com/r/Harini- Komaravelli/2659eee82e435d1b  ➢ 6 Yrs. of IT Experience in Manual and Automation testing.  WORK EXPERIENCE  QA Analyst  Oracle  Test Analyst at Oracle, Hyderabad  Infosys Ltd -  Hyderabad, Telangana -  November 2011 to February 2016  Hyderabad from Nov 2011 to Feb17 2016 ➢ Worked in Tata Consultancy Services, Hyderabad from Feb 24 to Apr 11 2017 ➢ Currently working as a Test Analyst at Oracle, Hyderabad  QA Analyst with 6 years of IT experience  Oracle  EDUCATION  MCA  Osmania University  B.Sc. in Computer Science  Osmania University  SKILLS  Functional Testing, Blue Prism, Qtp  ADDITIONAL INFORMATION  Area of Expertise:  ➢ Familiar with Agile Methodologies. ➢ Having knowledge in Energy (Petroleum) & Health Care domains. ➢ Involved in preparation of Test Scenarios. ➢ Preparing Test Data for the test cases.  https://www.indeed.com/r/Harini-Komaravelli/2659eee82e435d1b?isid=rex-download&ikw=download-top&co=IN https://www.indeed.com/r/Harini-Komaravelli/2659eee82e435d1b?isid=rex-download&ikw=download-top&co=IN   ➢ Experienced in development and execution of Test cases effectively. ➢ Experienced in Functional testing, GUI testing, Smoke testing, Regression testing and Integration Testing ➢ Experienced in doing Accessibility testing of an application ➢ Ability to understand user Requirements, Functional and Design specifications. ➢ Good knowledge of SDLC and STLC processes. ➢ Deciding the Severity and Priority of bugs. ➢ Experience in using Microsoft Test Manager & Oracle Test Manager as Test Management Tools. ➢ Having good experience in testing windows based & web based applications. ➢ Involved in Client Interactions for reviews, issues and for any clarifications. ➢ Web Services Testing ➢ Writing Test Scripts in QTP, Testcomplete. ➢ Creating Object Repositories and Function Libraries in QTP. ➢ Enhanced QTP scripts using VB Script. ➢ Strong experience in working with Blue Prism tool ➢ Worked on different Environments like Windows Application & Web Application  Technical Skills:  ❑ Test Automation Tools: Blue Prism, QTP 10.0, Testcomplete ❑ Test Management Tool: Microsoft Test Manager, Oracle Test Manager & JIRA ❑ Databases: Oracle 10g, SQL Server.  ❑ Operating Systems: Windows 7  Project 1: Title: Cadence Client: Baker Hughes  Technologies: Microsoft Visual Studio and Microsoft Team Foundation Server  Client Background: An oilfield services company delivering focused efforts on shale gas and other oilfield services. It provides services, tools and software for drilling and formation evaluation, well completion, production management, seismic data collection and interpretation.  Project Description: AUT (Application under test) is the next generation revolutionary, robust, easy to use scalable well site data acquisition processing and interpretation system for Client's Drilling Services to deliver services that meets cross divisional business requirements consistently.  Project 2:  Description: Paragon supports your entire care team with one tool that your clinicians need to help deliver the best patient care. Designed by physicians, nurses, pharmacists and mid level providers that have a first-hand understanding of clinical workflow needs, Paragon clinical applications allow your caregivers to focus on what matters most; spending time caring for patients. Since Paragon is fully-integrated across all applications and built around a single patient database, information    entered anywhere in the system is immediately available to the entire care team. Immediate access not only helps clinicians make better treatment decisions - it also helps promote patient safety. Paragon offers a broad suite of multidisciplinary clinical software solutions together with anytime, anywhere access to the complete patient record.  Responsibilities:  • Performed Smoke testing and Regression testing. • Involved in Generating and Executing Test Script using Quick Test Pro & Blue Prism • Usability and User Interface Testing. • Involved in Defect tracking and reporting the bugs using TFS • Participated in frequent walk-through meetings with Internal Quality Assurance groups and with development groups. • Participated in client calls and clarifying the doubts by having AT&T sessions • Involved in functional, regression and smoke testing to validate the application data changes done in windows application • Certifying the build status by running the scripts as part of smoke testing  Project 3:  Description: Food & Beverages R&A: Easily manage business across multiple locations while reducing IT cost and complexity. Cloud-based point-of-sale (POS) solutions enable centralized enterprise management with lower upfront costs and a smaller footprint.  Responsibilities:  • Performed Functional testing and Regression testing. • Involved in Generating and Executing Test Scripts using Blue Prism tool and Open script • Involved in preparing bots using Blue Prism tool. • Accessibility testing of the web application • Involved in Defect tracking and reporting the bugs using JIRA • WebServices testing by calling API's to export the data",)
for ents in doc.ents:
    print(ents.text, "-->>>>", ents.label_)

Harini Komaravelli -->>>> Name
Test Analyst -->>>> Designation
Oracle -->>>> Companies worked at
Hyderabad -->>>> Location
Hyderabad -->>>> Location
6 Yrs -->>>> Years of Experience
Manual and Automation testing. -->>>> Designation
QA Analyst -->>>> Designation
Oracle -->>>> Companies worked at
Test Analyst -->>>> Designation
Oracle -->>>> Companies worked at
Hyderabad -->>>> Location
Infosys Ltd -->>>> Companies worked at
Hyderabad -->>>> Location
Hyderabad -->>>> Location
Test Analyst -->>>> Designation
Oracle -->>>> Companies worked at
Hyderabad -->>>> Location
QA Analyst -->>>> Designation
6 years -->>>> Years of Experience
Oracle -->>>> Companies worked at
MCA -->>>> Degree
Osmania University -->>>> College Name
B.Sc. in Computer Science -->>>> Degree
Osmania University -->>>> College Name
Functional Testing, Blue Prism, Qtp -->>>> Skills
